In [1]:
%matplotlib inline
import time
import math
import warnings
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
import pystan 
import pandas as pd
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import pickle
from scipy import stats
from tqdm.notebook import tqdm as tqdm
from matplotlib.patches import Ellipse
from scipy.special import beta as B
from concurrent.futures import ProcessPoolExecutor

In [2]:
Iteration = 100000000
IterationDevide = int(Iteration/500000)

In [3]:
def PackageSize():
    return 1000 ###(mL)

In [4]:
def SEEDS(seed):
    seeds = seed
    if seeds > 2**32 - 1:
        seeds = seeds/0.9 - 2**32
        if seeds > 2**32 - 1:
            seeds = seeds/0.9 - 2**32
            if seeds > 2**32 - 1:
                seeds = seeds/0.9 - 2**32
                if seeds > 2**32 - 1:
                    seeds = seeds/0.9 - 2**32
                    if seeds > 2**32 - 1:
                        seeds = seeds/0.9 - 2**32
                        if seeds > 2**32 - 1:
                            seeds = seeds/0.9 - 2**32
    np.random.seed(round(seeds))
    DecidedSeed = np.random.randint(0, 2**32)
    return DecidedSeed

In [5]:
def PERT(a,b,c,seed):
    np.random.seed(SEEDS(seed))
    return a+(c-a)*np.random.beta(1+4*(b-a)/(c-a), 1+4*(c-b)/(c-a), size=1)

In [6]:
def Prevalence_InitialCount(seed):
    np.random.seed(SEEDS(seed))
    Classification_ID = np.random.randint(0,2)
    seed = seed + 1
    if Classification_ID == 0:
        Pre_P = 0.0222146854332712
        X = [0.4, 1, 10, 100, 1000, 10000]
        P = [0, 0.127627287, 0.82198086, 0.904286094, 0.941356302, 1]
        np.random.seed(SEEDS(seed))
        p = np.random.uniform(np.min(P), np.max(P))
        seed = seed + 1
        for i in range(len(P)):
            if i==np.max(range(len(P))):
                p = p
            else:
                    if ((P[i]<= p)&(p<P[i+1])):
                        if X[i]==0:
                            np.random.seed(SEEDS(seed))
                            N_initial = np.random.uniform(X[i], X[i+1])
                            seed = seed + 1
                        else:
                            np.random.seed(SEEDS(seed))
                            N_initial = 10**np.random.uniform(np.log10(X[i]), np.log10(X[i+1]))
                            seed = seed + 1
        

    if Classification_ID == 1:
        Pre_P = 0.0323275862068966
        X = [5.09304E-08, 1.89123E-07, 7.29106E-07, 2.81085E-06, 1.08364E-05, 4.02394E-05, 0.000155131, 0.000620902, 0.002305636, 0.008888679, 0.033006893, 0.127248058, 0.490566259, 1.821650746, 6.764451044, 28.10847015, 104.3769617, 402.3936911, 1494.233968]
        P = [0.00000939410713882217, 0.000028403016635518, 0.0000561662968647522, 0.000143298898365032, 0.000608585201471365, 0.0017886907659385, 0.00697796127721843, 0.0163545637122337, 0.0602855872788407, 0.14951524425208, 0.415249566701753, 0.775943942187806, 0.948255294880887, 0.974127647440443, 1, 1, 1, 1, 1]
        np.random.seed(SEEDS(seed))
        p = np.random.uniform(np.min(P), np.max(P), size=1)
        seed = seed + 1
        
        for i in range(len(P)):
            if i==np.max(range(len(P))):
                p = p
            else:
                    if ((P[i]<= p)&(p<P[i+1])):
                        if X[i]==0:
                            np.random.seed(SEEDS(seed))
                            N_initial = np.random.uniform(X[i], X[i+1])
                            seed = seed + 1
                        else:
                            np.random.seed(SEEDS(seed))
                            N_initial = 10**np.random.uniform(np.log10(X[i]), np.log10(X[i+1]))
                            seed = seed + 1
        

    return Pre_P, N_initial # (-), cfu/mL

In [7]:
Prevalence_InitialCount(5)

(0.0222146854332712, 1.1369711988855444)

In [8]:
def MPD(seed):
    np.random.seed(SEEDS(seed))
    Max = np.random.uniform(7.5, 8.5, size=1)
    return Max[0] #cfu/mL

In [9]:
root_growth_rate_mean = pickle.load(open('../Growth/growth rate mean.pkl', 'rb'))
sm_µ_max = pickle.load(open('../Growth/stan_model_µ_max.pkl', 'rb'))
fit_nuts_µ_max = pickle.load(open('../Growth/fit_nuts_µ_max.pkl', 'rb'))
samples_µ_max = fit_nuts_µ_max.extract(permuted=True)
Temp_ref_growth = 8.518181818181818


def Para_growth(seed):
    np.random.seed(SEEDS(seed))
    rcn = np.random.randint(0,len(samples_µ_max ["lp__"]))
    mu_ref = samples_µ_max["mu_ref"][rcn]
    T_min = samples_µ_max["T_min"][rcn]
    s_mu = samples_µ_max["s_mu"][rcn]
    return [mu_ref, T_min, s_mu] 

In [10]:
def root_mu_median(Temp):
    for k in range(len(root_growth_rate_mean[:,0])-1):
        if ((root_growth_rate_mean[k,0] <= Temp)&(Temp < root_growth_rate_mean[k+1,0])):
            out = (root_growth_rate_mean[k,1]+(root_growth_rate_mean[k+1,1]-root_growth_rate_mean[k,1])*(Temp-root_growth_rate_mean[k,0]))
    return out

In [11]:
root_mu_median(-9)

-0.07915413223562695

In [12]:
def OnFarm(seed):
    Classification_ID = 0
    if Classification_ID == 0:
        X = [-0.25,0.25,0.75,1.25,1.75,2.25,2.75,3.25,3.75,4.25,4.75,5.25,5.75,6.25,6.75,7.25,7.75,8.25,8.75,9.25,9.75,10.25,10.75,11.25,11.75,12.25]
        P = [0,0.000792016,0.000792016,0.003009663,0.011246634,0.062886108,0.150641533,0.355140187,0.555837161,0.722794234,0.809282433,0.866624426,0.909710122,0.941707588,0.962141613,0.982734041,0.989703786,0.993980675,0.996515128,0.998732774,0.999207984,0.999683193,0.999841597,0.999841597,0.999841597,1.]
        np.random.seed(SEEDS(seed))
        p = np.random.uniform(np.min(P), np.max(P))
        seed = seed + 1
        for i in range(len(P)):
            if i==np.max(range(len(P))):
                p = p
            else:
                if ((P[i]<= p)&(p<P[i+1])):
                    np.random.seed(SEEDS(seed))
                    Temp = np.random.uniform(X[i], X[i+1])
                    seed = seed + 1

        np.random.seed(SEEDS(seed))
        Time = PERT(1.5, 12, 36, seed)[0]

    return Temp, Time

In [13]:
def Unc_heat_temp(seed):
    np.random.seed(SEEDS(seed))
    CHOICE_UNC = np.random.randint(0,4)
    unc_list = [0.05, 0.25, 0.5, 1]
    unc = unc_list[CHOICE_UNC]
    return unc

In [14]:
def Temp_uncertainty(Temp, seed):
    
    Classification_ID = 0
    if Classification_ID == 0:
        a = Temp-Unc_heat_temp(seed)
        b = Temp
        c = Temp+Unc_heat_temp(seed)
        Temp_unc = PERT(a, b, c, seed)
    return Temp_unc[0]

In [15]:
logD_mean = pickle.load(open('../reduction/LogD mean.pkl', 'rb'))
sm_metadata_D = pickle.load(open('../reduction/stan_model_D.pkl', 'rb'))
fit_nuts_metadata_D = pickle.load(open('../reduction/fit_nuts_metadata_D.pkl', 'rb'))
samples_metadata_D = fit_nuts_metadata_D.extract(permuted=True)
Temp_ref_heat = 62.33197115384615


def Para_reduction(seed):
    np.random.seed(SEEDS(seed))
    rcn = np.random.randint(0,len(samples_metadata_D["lp__"]))
    z = samples_metadata_D["z"][rcn]
    LogD_ref = np.log10(samples_metadata_D["D_ref"][rcn])
    s_LogD = samples_metadata_D["s_logD"][rcn]
    return [z, LogD_ref, s_LogD, rcn] #SecondPara: [z, LogD_ref, beta_ref, rcn]

In [16]:
def HeatTimeVariability(sec, val_rate, seed):
    
    OUT = PERT(sec*(1-val_rate), sec, sec*(1+val_rate), seed)[0]
    return OUT

In [17]:
def Heat(seed):
    np.random.seed(SEEDS(seed))
    Classification_ID = np.random.randint(0,4)
    if Classification_ID == 0:
        Temp_heat = 72.
        Time = 15.
    if Classification_ID == 3:
        Temp_heat = 75.
        Time = 20.
    if Classification_ID == 4:
        Temp_heat = 85.
        Time = 20.
    if Classification_ID == 5:
        Temp_heat = 89.
        Time = 1.
    if Classification_ID == 2:
        Temp_heat = 74.
        Time = 20.
    if Classification_ID == 1:
        Temp_heat = 72.5
        Time = 20.
        
    return [Temp_heat, Time] #ºC, sec

In [18]:
def Distributing(seed):
    np.random.seed(SEEDS(seed))
    Classification_ID = 0
    if Classification_ID == 0:
        Temp = np.random.normal(6.66, 1.64)
        seed = seed + 1
        np.random.seed(SEEDS(seed))
        Time = 4.33*np.random.weibull(1.98)
    return Temp, Time #(h)

In [19]:
def Val_ratio(seed):
    np.random.seed(SEEDS(seed))
    CHOICE_VAL = np.random.randint(0,4)
    ratio_list = [0.01, 0.05, 0.1, 0.25]
    ratio = ratio_list[CHOICE_VAL]
    return ratio

In [20]:
def Retail(seed):
    np.random.seed(SEEDS(seed))
    
    Classification_ID = 0
    if Classification_ID == 0:
        Temp = np.random.normal(4.98, 2.9)
        seed = seed + 1
        np.random.seed(SEEDS(seed))
        X = [0, 5, 15, 25, 35, 45, 55, 65, 75, 85, 95, 105]
        P = [0, 0.019811321, 0.443396226, 0.717924528, 0.85,0.916037736,0.950943396,0.969811321,0.982075472,0.99245283,0.997169811,1.]
        p = np.random.uniform(np.min(P), np.max(P), size=1)
        seed = seed + 1
        np.random.seed(SEEDS(seed))
        for i in range(len(P)):
            if i==np.max(range(len(P))):
                p = p
            else:
                if ((P[i]<= p)&(p<P[i+1])):
                    Time = np.random.uniform(X[i], X[i+1])
    return Temp, Time #(h)

In [21]:
def Domestic(seed):
    
    np.random.seed(SEEDS(seed))
    Classification_ID = 0
    if Classification_ID == 0:
        P = [0., 0.25, 0.25+0.19, 0.25+0.19+0.05, 0.25+0.19+0.05+0.51]
        p = np.random.uniform(np.min(P), np.max(P), size=1)
        seed = seed + 1
        np.random.seed(SEEDS(seed))
        mean = [7.57, 6.31, 6.69, 8.40]
        sd = [2.95, 2.66, 3.29, 3.00]
        for i in range(len(P)):
            if i==np.max(range(len(P))):
                p = p
            else:
                if ((P[i]<= p)&(p<P[i+1])):
                    Temp = np.random.normal(mean[i], sd[i])
                    seed = seed + 1
                    np.random.seed(SEEDS(seed))
        X = [0, 24, 48, 72, 96, 120]
        P = [0, .23, .74, .96, .98, 1.]
        p = np.random.uniform(np.min(P), np.max(P))
        seed = seed + 1
        np.random.seed(SEEDS(seed))
        for i in range(len(P)):
            if i==np.max(range(len(P))):
                p = p
            else:
                if ((P[i]<= p)&(p<P[i+1])):
                    Time = np.random.uniform(X[i], X[i+1])
    return Temp, Time #(h)

In [22]:
def Size_serve(seed):
    np.random.seed(SEEDS(seed))
    
    Classification_ID = 0
    if Classification_ID == 0:
        X = [0./1.025, 244/1.025, 245/1.025, 488/1.025, 732/1.025]
        P = [0., 0.50, .75,  .95, 1.0]
        p = np.random.uniform(np.min(P), np.max(P))
        seed = seed + 1
        np.random.seed(SEEDS(seed))
        for i in range(len(P)):
            if i==np.max(range(len(P))):
                p = p
            else:
                if ((P[i]<= p)&(p<P[i+1])):
                    Sizeserve = np.random.uniform(X[i], X[i+1])
    return Sizeserve   #mL

In [23]:
def Para_R(seed):
    np.random.seed(SEEDS(seed))
    
    P = [0, 0.8228571428571428, 0.8285714285714285, 1.0] #(3.60*10**8, 2.5*10**6, 7.5*10**7)
    p = np.random.uniform(np.min(P), np.max(P), size=1)
    seed = seed + 1
    np.random.seed(SEEDS(seed))
    if ((p>=P[0])&(p<P[1])):
        PR = 7.9*10**-12
        PR = 10**(np.random.normal(-14.11, 1.62))
        CLASS = 1
    if ((p>=P[1])&(p<P[2])):
        PR = 2.01*10**-9
        PR = 10**(np.random.normal(-11.70, 1.62))
        CLASS = 2
    if ((p>=P[2])&(p<P[3])):
        PR = 1.49*10**-10
        PR = 10**(np.random.normal(-12.83, 1.62))
        CLASS = 3
    return PR, CLASS

In [24]:
Servings_per_year = 3.6*10**8

In [25]:
def QMRA_simulations(seed):
    warnings.simplefilter('ignore', RuntimeWarning)
    prevalence_initialcount = Prevalence_InitialCount(seed)
    seed = seed + 5
    np.random.seed(SEEDS(seed))
    prevalence  = prevalence_initialcount[0]
    prevalence_or_not = np.random.binomial(n = 1, p = prevalence )
    seed = seed + 5
    np.random.seed(SEEDS(seed))
    package_size = PackageSize()
    
    if prevalence_or_not == 1:
        
        mpd = MPD(seed)
        seed = seed + 5
        
        logC0  = np.log10(prevalence_initialcount[1])   #log cfu/mL
        para_growth = Para_growth(seed)   #[mu_ref, T_min, s_mu] 
        seed = seed + 5
        np.random.seed(SEEDS(seed))
        mu_ref = para_growth[0]
        temp_min = para_growth[1]
        s_mu = para_growth[2]
        
        onfarm = OnFarm(seed)
        seed = seed + 5
        np.random.seed(SEEDS(seed))
        temp_pre  = onfarm[0]
        time_pre  = onfarm[1]   #(h)
        
        root_mu0_pre = (mu_ref**0.5)*(((temp_pre -temp_min)/(Temp_ref_growth-temp_min)))
        sigma_mu_root = np.random.normal(0, s_mu)
        seed = seed + 5
        np.random.seed(SEEDS(seed))
        root_mu_pre = root_mu0_pre + sigma_mu_root
        root_mu_pre = np.max([root_mu_pre, 0])
        mu_pre = root_mu_pre**2.
        mu_pre_variability  = root_mu_pre - np.max([root_mu_median(temp_pre), 0])
                
        logC_pre  = np.min((np.maximum(logC0, mpd), logC0 + mu_pre*time_pre))
        seed = seed + 5
        np.random.seed(SEEDS(seed))
    
        para_redcution = Para_reduction(seed) #[z, LogD_ref, s_LogD, rcn] 
        seed = seed + 5
        np.random.seed(SEEDS(seed))
        z = para_redcution[0]
        logD_ref = para_redcution[1]
        s_LogD = para_redcution[2]
        rcn_reduction = para_redcution[3]
    
        heat = Heat(seed)   #[temp_heat [0], Time[0]] #ºC, sec
        seed = seed + 5
        np.random.seed(SEEDS(seed))
        temp_mean_heat  = heat[0]
        time_mean_heat  = heat[1]
        
        temp_heat  = Temp_uncertainty(temp_mean_heat, seed)
        seed = seed + 5
        np.random.seed(SEEDS(seed))
        val_ratio = Val_ratio(seed)
        seed = seed + 5
        np.random.seed(SEEDS(seed))
        time_heat  = HeatTimeVariability(time_mean_heat, val_ratio, seed)
        seed = seed + 5
        np.random.seed(SEEDS(seed))
        
        heat_time_variability  = time_heat - time_mean_heat 
        heat_temp_uncertainty  = temp_heat - temp_mean_heat 
    
        logD0 = -(temp_heat - Temp_ref_heat)/z + logD_ref
        logD = np.random.normal(logD0, s_LogD)
        seed = seed + 5
        np.random.seed(SEEDS(seed))
        D = 10**logD
        
        for k in range(len(logD_mean[:,0])-1):
            if ((logD_mean[k,0] <= temp_heat )&(temp_heat  < logD_mean[k+1,0])):
                logD_variability  = logD - (logD_mean[k,1]+(logD_mean[k+1,1] - logD_mean[k,1])*(temp_heat - logD_mean[k,0]))
    
        logC_heat = logC_pre - (time_heat/D/60)
        C_heat = 10**(logC_heat)
        N_heat_packed = np.random.poisson(C_heat*package_size)
        seed = seed + 5
        np.random.seed(SEEDS(seed))
        C_heat_packed = N_heat_packed/package_size
        logC_heat_packed = np.log10(C_heat_packed)
        
        distributing = Distributing(seed)
        seed = seed + 5
        np.random.seed(SEEDS(seed))
        temp_dist = distributing[0]
        time_dist = distributing[1]
        
        root_mu0_dist = (mu_ref**0.5)*(((temp_dist -temp_min)/(Temp_ref_growth-temp_min)))
        sigma_mu_root = np.random.normal(0, s_mu)
        seed = seed + 5
        np.random.seed(SEEDS(seed))
        root_mu_dist =  root_mu0_dist + sigma_mu_root
        root_mu_dist = np.max([root_mu_dist, 0])
        mu_dist = root_mu_dist**2.
        mu_dist_variability  = root_mu_dist - np.max([root_mu_median(temp_dist), 0])
        
        cap = round(package_size*10**mpd)
        seed = seed + 5
        np.random.seed(SEEDS(seed))
        if N_heat_packed == 0:
            N_dist = 0
        elif N_heat_packed >= cap:
            N_dist = N_heat_packed
        else:
            delta_N = np.random.negative_binomial(N_heat_packed, 10**(-mu_dist*time_dist), size=1)[0]
            N_dist = np.min([N_heat_packed + delta_N, cap])
        seed = seed + 5
        np.random.seed(SEEDS(seed))
        C_dist = N_dist/package_size
        logC_dist = np.log10(C_dist)
        
        retail = Retail(seed)
        seed = seed + 5
        np.random.seed(SEEDS(seed))
        temp_retail  = retail[0]
        time_retail  = retail[1]
        
        root_mu0_retail = (mu_ref**0.5)*(((temp_retail -temp_min)/(Temp_ref_growth-temp_min)))
        sigma_mu_root = np.random.normal(0, s_mu)
        seed = seed + 5
        np.random.seed(SEEDS(seed))
        root_mu_retail = root_mu0_retail + sigma_mu_root
        root_mu_retail = np.max([root_mu_retail, 0])
        mu_retail = root_mu_retail**2.
        mu_retail_variability  = root_mu_retail - np.max([root_mu_median(temp_retail), 0])
        
        cap = round(package_size*10**mpd)
        seed = seed + 5
        np.random.seed(SEEDS(seed))
        if N_dist == 0:
            N_retail = 0
        elif N_dist >= cap:
            N_retail = N_dist
        else:
            N_retail = N_dist + np.random.negative_binomial(N_dist, 10**(-mu_retail*time_retail), size=1)[0]
            N_retail = np.min([N_retail, cap])
        seed = seed + 5
        np.random.seed(SEEDS(seed))
        C_retail = N_retail/package_size
        logC_retail = np.log10(C_retail)
        
        domestic = Domestic(seed)
        seed = seed + 5
        np.random.seed(SEEDS(seed))
        temp_domestic = domestic[0]
        time_domestic = domestic[1]
    
        root_mu0_domestic = (mu_ref**0.5)*(((temp_domestic -temp_min)/(Temp_ref_growth-temp_min)))
        sigma_mu_root = np.random.normal(0, s_mu)
        seed = seed + 5
        np.random.seed(SEEDS(seed))
        root_mu_domestic = root_mu0_domestic + sigma_mu_root
        root_mu_domestic = np.max([root_mu_domestic, 0])
        mu_domestic = root_mu_domestic**2.
        mu_domestic_variability  = root_mu_domestic - np.max([root_mu_median(temp_domestic), 0])
        
        cap = round(package_size*10**mpd)
        seed = seed + 5
        np.random.seed(SEEDS(seed))
        if N_retail == 0:
            N_domestic = 0
        elif N_retail >= cap:
            N_domestic = N_retail
        else:
            N_domestic = N_retail + np.random.negative_binomial(N_retail, 10**(-mu_domestic*time_domestic), size=1)[0]
            N_domestic = np.min([N_domestic, cap])
        seed = seed + 5
        np.random.seed(SEEDS(seed))
        
        C_domestic = N_domestic/package_size
        logC_domestic = np.log10(C_domestic)
            
        size_serve  = Size_serve(seed)
        seed = seed + 5
        np.random.seed(SEEDS(seed))
        N_dose = np.random.poisson(C_domestic*size_serve)
        seed = seed + 5
        np.random.seed(SEEDS(seed))
        logN_dose = np.log10(N_dose)
        
        Para_r = Para_R(seed)
        para_r = Para_r[0]
        subject_class = Para_r[1]
        seed = seed + 5
        np.random.seed(SEEDS(seed))
        
        P_serve = prevalence_or_not*np.float64(1-np.exp(-para_r*N_dose))
        
        Ill_or_not  = np.random.binomial(n=1, p = P_serve)
    
    if prevalence_or_not == 0:
        logC0 = -float('inf')
        mpd = 0
        logC_pre = -float('inf')
        logC_heat_packed = -float('inf')
        logC_heat = -float('inf')
        logC_dist = -float('inf')
        logC_retail = -float('inf')
        logC_domestic = -float('inf')
        logN_dose = -float('inf')
        P_serve = 0
        Ill_or_not = 0
        temp_pre = 0
        time_pre = 0
        temp_heat = 0
        time_heat = 0
        temp_mean_heat = 0
        time_mean_heat = 0
        heat_time_variability = 0
        heat_temp_uncertainty = 0
        temp_dist = 0
        time_dist = 0
        time_retail = 0
        temp_retail = 0
        time_domestic = 0
        temp_domestic = 0
        mu_pre_variability = 0
        mu_dist_variability = 0
        mu_retail_variability = 0
        mu_domestic_variability = 0
        logD_variability = 0
        size_serve = 0
        para_r = 0
        subject_class = 0
    return [prevalence, logC0, logC_pre, logC_heat, logC_dist, logC_retail, logC_domestic, logN_dose, P_serve, prevalence_or_not, Ill_or_not, temp_pre, time_pre, temp_heat, time_heat, temp_mean_heat, time_mean_heat, heat_time_variability, heat_temp_uncertainty, temp_dist, time_dist, time_retail, temp_retail, time_domestic, temp_domestic, mu_pre_variability, mu_dist_variability, mu_retail_variability, mu_domestic_variability, logD_variability, size_serve, para_r, subject_class, logC_heat_packed, mpd]

In [26]:
start = time.time()
sms_multi =[]
for q in tqdm(range(30, 60, 1), desc='Progress', leave=False):
    CopyNum = q
    for k in tqdm(range(IterationDevide),desc='Progress', leave=False):
        ns = range(CopyNum*Iteration+500000*k, CopyNum*Iteration+500000*(k+1))
        with ProcessPoolExecutor(6) as e:
            ret = e.map(QMRA_simulations, ns)
        sms_multi_k = [r for r in ret]
        for i in range(500000):
            if sms_multi_k[i][10]==1:
                print("Copy Num: "+str(CopyNum)+" !!! illness !!!")
        with open('pickle output/QMRA output'+str(k+CopyNum*Iteration/500000)+'.pkl', 'wb') as g:
            pickle.dump(sms_multi_k, g)
    end = time.time()
    delta = end - start
    print('処理時間:{}hours'.format(round(delta/60/60,3)))  

Progress:   0%|          | 0/30 [00:00<?, ?it/s]

Progress:   0%|          | 0/200 [00:00<?, ?it/s]

処理時間:4.609hours


Progress:   0%|          | 0/200 [00:00<?, ?it/s]

Copy Num: 31 !!! illness !!!
処理時間:9.206hours


...

処理時間:129.04hours


Progress:   0%|          | 0/200 [00:00<?, ?it/s]

処理時間:133.655hours


Progress:   0%|          | 0/200 [00:00<?, ?it/s]

処理時間:138.268hours
